# Potential: Force Field Parameters

Force field parameter management and storage.

## Structure

- Atom types: mass, LJ parameters
- Bond types: spring constants
- Angle types: equilibrium angles
- Dihedral types: Fourier coefficients

## See Also

- [Force Field Parameters](forcefield_parameters.ipynb)